In [11]:
# ============================
# IMPORTAR LIBRERÍAS NECESARIAS
# ============================

import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model

In [12]:
# ============================
# DEFINIR RUTA DEL DATASET
# ============================

path = '/kaggle/input/face-expression-recognition-dataset/images/validation/'

In [13]:
# ============================
# LISTAR ARCHIVOS DE IMÁGENES
# ============================

image_names = os.listdir(path)
print(f"Total de imágenes encontradas: {len(image_names)}")

Total de imágenes encontradas: 7


In [14]:
# ============================
# CARGAR Y PREPROCESAR IMÁGENES
# ============================

# Lista para almacenar todas las imágenes
images = []
image_paths = []

# Recorrer cada subcarpeta ('angry', 'happy', 'surprise')
for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    
    if os.path.isdir(folder_path):
        # Recorrer cada imagen dentro de la subcarpeta
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            
            # Cargar imagen solo si es un archivo
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            
            images.append(img_array)
            image_paths.append(img_path)  # Guardar también el path de la imagen

# Convertir la lista de imágenes a array numpy
images = np.vstack(images)

print(f"Total imágenes cargadas: {len(images)}")

Total imágenes cargadas: 7066


In [16]:
# ============================
# CARGAR MODELO VGG16
# ============================

# Modelo VGG16 preentrenado sin la capa de clasificación final
base_model = VGG16(weights=None)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)

In [ ]:
# ============================
# EXTRAER VECTORES DE CARACTERÍSTICAS
# ============================

features = model.predict(images, batch_size=32, verbose=1)
print(f"Forma de los vectores de características: {features.shape}")

 33/221 ━━━━━━━━━━━━━━━━━━━━ 28:28 9s/step

In [ ]:
# ============================
# REDUCCIÓN DE DIMENSIONALIDAD
# ============================

# Aplicar PCA para reducir dimensiones a 50 componentes
pca = PCA(n_components=50)
features_pca = pca.fit_transform(features)

print(f"Forma después de PCA: {features_pca.shape}")

In [ ]:
# ============================
# APLICAR CLUSTERING K-MEANS
# ============================

# Definir el número de clusters
n_clusters = 7

# Crear y ajustar modelo de clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(features_pca)

print(f"Etiquetas de clustering asignadas: {np.unique(clusters)}")

In [ ]:
# ============================
# EVALUAR CALIDAD DEL CLUSTERING
# ============================

silhouette_avg = silhouette_score(features_pca, clusters)
print(f"Coeficiente de silueta (Silhouette Score): {silhouette_avg:.4f}")

In [ ]:
# ============================
# VISUALIZAR EJEMPLOS DE CLUSTERS
# ============================

def show_cluster_examples(cluster_label, n_examples=5):
    """
    Muestra ejemplos de imágenes de un cluster específico.
    
    Args:
    - cluster_label: Número de cluster que quieres visualizar.
    - n_examples: Cantidad de imágenes a mostrar.
    """
    # Encontrar los índices de las imágenes que pertenecen al cluster solicitado
    indices = np.where(clusters == cluster_label)[0]
    
    plt.figure(figsize=(15, 3))
    
    for i, idx in enumerate(indices[:n_examples]):
        img_path = image_paths[idx]  # 🛠️ CORREGIDO: usar la lista real de rutas
        img = image.load_img(img_path, target_size=(224, 224))
        
        plt.subplot(1, n_examples, i + 1)
        plt.imshow(img)
        plt.axis('off')
    
    plt.suptitle(f"Ejemplos del Cluster {cluster_label}", fontsize=16)
    plt.show()

In [ ]:
# ============================
# MOSTRAR EJEMPLOS DE CLUSTERS
# ============================

# Cambia el número para ver otros clusters (0,1,2,...)
show_cluster_examples(cluster_label=1, n_examples=5)